Primary Objective: complete all the steps below and return the risk analysis of your seven (7) stock portfolio
against the S&P500 (SPY), Russell 2000 (IWM), and the Dow Jones Industrial Average (DIA).

Requirements:
1. Create a table showing constituent (stocks) risk analysis in the equal-weight portfolio analysis as of the current date.
    a. Column 1 – Ticker
    b. Column 2 – Portfolio Weight (equally weighted)
    c. Column 3 – Annualized Volatility (using trailing 3-months)
    d. Column 4 – Beta against SPY (using trailing 12-months)
    e. Column 5 – Beta against IWM (using trailing 12-months)
    f. Column 6 – Beta against DIA (using trailing 12-months)
    g. Column 7 – Average Weekly Drawdown (52-week Low minus 52-week High) / 52-week High
    h. Column 8 – Maximum Weekly Drawdown (52-week Low minus 52-week High) / 52-week High
    i. Column 9 – Total Return (using trailing 10-years)
    j. Column 10 – Annualized Total Return (using trailing 10-years)
2. Create a table showing Portfolio Risk against the three ETFs:
    a. Column 1 – ETF Ticker
    b. Column 2 – Correlation against ETF
    c. Column 3 – Covariance of Portfolio against ETF
    d. Column 4 – Tracking Errors (using trailing 10-years)
    e. Column 5 – Sharpe Ratio (using current risk-free rate)
    f. Column 6 – Annualized Volatility (252 days) Spread (Portfolio Volatility – ETF Volatility)
3. Create a correlation matrix showing the correlations between the equal-weighted portfolio, 3 ETFs, and your 7 stocks.
4. Output these two tables and the matrix into a PDF file. You must submit both your Python code file and your PDF output file.


In [3]:
#For package verification
#pip install holidays
#pip install yfinance
#7 stocks picked are MSFT, AAPL, GOOG, AMD, BRK.A, JPM, and DAL
#This part was completed thanks to the help of following documentations:
#https://algotrading101.com/learn/yfinance-guide/
#https://docs.python.org/3/library/datetime.html
#https://stackoverflow.com/questions/2394235/detecting-a-us-holiday
#https://numpy.org/doc/stable/reference/generated/numpy.cov.html
#1. Average Drawdown vs Maximum Drawdown
#2. Volatility - price or returns?
#3. Covariance function - what's the catch?
#4. Tracking Errors? of what?


import yfinance as yf
import pandas as pd
import numpy as np
import datetime as dt
import holidays
us_holidays = holidays.US()

In [4]:
#Create some useful function for calculation needs
def open_date(date):
    #Check if the market is open on this specific date, and if not,
    #Move to the next available date
    move = 0
    
    #Market closes on weekends and holidays, and if holiday is on Saturday, close on Friday,
    #if holiday in on Sunday, close the following Monday
    start_date = dt.datetime.strptime(date,'%Y-%m-%d')
    week_day = start_date.weekday()
    if week_day == 5:
        move += 2
    elif week_day == 6:
        if date in us_holidays:
            move+=2
        else:
            move+=1
    else:
        if date in us_holidays:
            move+=1
    return start_date + dt.timedelta(days=move)
    
def calculate_shares(stock, money, date, split=7):
    #Count the number of shares that can be bought for certain stock,
    #and gives the money remaining after buying those shares
    #Default split is 1/7 
    percentage = round(1/split * money,2)            
    start_date = open_date(date)
    end_date = start_date+dt.timedelta(days=1)
    current = stock.history(start=start_date,end=end_date).Close
    return percentage//current, percetage%current

def daily_return(stock,period):
    #Return a list of daily returns of a certain stock
    prices = yf.download(stock,period = period)
    adj_close = []
    rate_of_return = []
    for close_price in prices['Adj Close']:
        adj_close.append(close_price)
    for x in range(len(adj_close)):
        if x == 0:
            continue
        rate_of_return.append((adj_close[x] - adj_close[x-1])/adj_close[x-1])
    return rate_of_return

def volatility(stock, period):
    #Calculate market volatility based on the start and end date
    sd_return = np.std(daily_return(stock,period))
    #252 trading days a year to calculate annualized volatility from one day
    return sd_return*np.sqrt(252)

def calculate_beta(stock,benchmark,period):
    #calculate covariance using stock and index info
    stock_return = daily_return(stock,period)
    index_return = daily_return(benchmark,period)
    #I couldn't figure out why the covariance function does not work for the life of me, so I took it to my hands
    i = np.array(index_return)
    s = np.array(stock_return)
    ai = np.mean(i)
    sa = np.mean(s)
    i1 = i-ai #benchmark daily return - average return
    s1 = s-sa #stock daily return - average return 
    r = np.dot(i1,s1) #take the dot product of them to calculate covariance
    cov = r/len(i) #divide by n - 1
    sd_i = np.std(i)**2 #get the index variance
    #Ideally Covariance should form something like this from pythonm, but np.cov doesn't seem to work
    #           stock    benchmark
    # stock    stock_var  cov
    #benchmark cov      benchmark_var
    #and beta is calculated using covariance of stock and benchmark/variance of stock
    return cov/sd_i

def total_return(stock,period,annualized = 'n'):
    prices = yf.download(stock,period = period)
    return (prices['Adj Close'][-1] - prices['Adj Close'][0])/prices['Adj Close'][0])
    

In [16]:
#Initiate analysis
risk_analysis = pd.DataFrame(columns=['Ticker','Portfolio Weight','Annualized Volatility(Trailing 3-month)',
                                      'Beta vs SPY','Beta vs IWM', 'Beta vs DIA', 
                                      'Average Weekly Drawdown','Maximum Weekly Drawdown',
                                      'Total Return over 10 years (in %)','Annualized Total Return over 10 years (in %)'])
risk_analysis['Ticker'] = ['MSFT', 'AAPL', 'GOOG', 'AMD', 'BLK', 'JPM', 'DAL']
#While they are supposed to be equally weighted, because of how prices actually don't allow this to happen,
#I have adjusted the weight based on a whole share based on the 10-year date being tested,
for x in range(6):
    risk_analysis['Portfolio Weight'][x] = 14.3
risk_analysis['Portfolio Weight'][6] = 14.2

In [17]:
#Measuing volatility in terms of annualized percentage change
count = 0
for Ticker in risk_analysis['Ticker']:
    risk_analysis['Annualized Volatility(Trailing 3-month)'][count] = volatility(Ticker,'3mo')
    count+=1
    
#Measuring Beta vs SPY
#Beta is measured against a benchmark.
count = 0
for Ticker in risk_analysis['Ticker']:
    risk_analysis['Beta vs SPY'][count] = calculate_beta(Ticker,'SPY','1y')
    count+=1
    
#Measuring Beta vs IWM
#Beta is measured against a benchmark.
count = 0
for Ticker in risk_analysis['Ticker']:
    risk_analysis['Beta vs IWM'][count] = calculate_beta(Ticker,'IWM','1y')
    count+=1
    
#Measuring Beta vs DIA
#Beta is measured against a benchmark.
count = 0
for Ticker in risk_analysis['Ticker']:
    risk_analysis['Beta vs DIA'][count] = calculate_beta(Ticker,'DIA','1y')
    count+=1
    
#Find total return
count = 0
for Ticker in risk_analysis['Ticker']:
    risk_analysis['Total Return over 10 years (in %)'][count] = total_return(Ticker,'10y')*100
    count+=1
    
#Find annualized return uising trailing 10 year
count = 0
for Ticker in risk_analysis['Ticker']:
    risk_analysis['Annualized Total Return over 10 years (in %)'][count] = ((1+total_return(Ticker,'10y'))**(1/10)- 1)*100
    count+=1

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [18]:
risk_analysis

,Ticker,Portfolio Weight,Annualized Volatility(Trailing 3-month),Beta vs SPY,Beta vs IWM,Beta vs DIA,Average Weekly Drawdown,Maximum Weekly Drawdown,Total Return over 10 years (in %),Annualized Total Return over 10 years (in %)
0,MSFT,14.3,0.36626,1.214041,0.757613,1.274419,NaN,NaN,957.934175,26.602793
1,AAPL,14.3,0.402701,1.268715,0.837971,1.374925,NaN,NaN,701.658863,23.141605
2,GOOG,14.3,0.413948,1.279502,0.831237,1.345179,NaN,NaN,678.971635,22.786937
3,AMD,14.3,0.662817,2.100048,1.588613,2.105853,NaN,NaN,1531.884077,32.212246
4,BLK,14.3,0.375655,1.167993,0.839189,1.385933,NaN,NaN,332.384776,15.767597
5,JPM,14.3,0.309973,0.875159,0.648641,1.184793,NaN,NaN,304.713869,15.004493
6,DAL,14.2,0.521198,1.433404,1.285698,1.810444,NaN,NaN,205.46798,11.814098


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Ticker,Portfolio Weight,Annualized Volatility(Trailing 3-month),Beta vs SPY,Beta vs IWM,Beta vs DIA,Average Weekly Drawdown,Maximum Weekly Drawdown,Total Return over 10 years,Annualized Total Return over 10 years
0,MSFT,14.3,0.368983,1.214785,0.759825,NaN,NaN,NaN,NaN,NaN
1,AAPL,14.3,0.403381,1.270557,0.842615,NaN,NaN,NaN,NaN,NaN
2,GOOG,14.3,0.417315,1.279412,0.831713,NaN,NaN,NaN,NaN,NaN
3,AMD,14.3,0.667179,2.101766,1.593786,NaN,NaN,NaN,NaN,NaN
4,BLK,14.3,0.378137,1.167395,0.838708,NaN,NaN,NaN,NaN,NaN
5,JPM,14.3,0.304219,0.87267,0.643977,NaN,NaN,NaN,NaN,NaN
6,DAL,14.2,0.52529,1.433956,1.288221,NaN,NaN,NaN,NaN,NaN


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Ticker,Portfolio Weight,Annualized Volatility(Trailing 3-month),Beta vs SPY,Beta vs IWM,Beta vs DIA,Average Weekly Drawdown,Maximum Weekly Drawdown,Total Return over 10 years,Annualized Total Return over 10 years
0,MSFT,14.3,0.368983,1.214785,0.759825,1.276958,NaN,NaN,NaN,NaN
1,AAPL,14.3,0.403381,1.270557,0.842615,1.380351,NaN,NaN,NaN,NaN
2,GOOG,14.3,0.417315,1.279412,0.831713,1.345531,NaN,NaN,NaN,NaN
3,AMD,14.3,0.667179,2.101766,1.593786,2.111204,NaN,NaN,NaN,NaN
4,BLK,14.3,0.378137,1.167395,0.838708,1.385153,NaN,NaN,NaN,NaN
5,JPM,14.3,0.304219,0.87267,0.643977,1.179114,NaN,NaN,NaN,NaN
6,DAL,14.2,0.52529,1.433956,1.288221,1.812875,NaN,NaN,NaN,NaN


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [141]:

r

0.00776421711196627

1.8872586057514427

1.8872586057514424